# Validate BoM

The notebook/script reads in the df file from sharepoint based on the existing live BoM for the chosen project.  It will correct the part numbers being used by matching with catia   
and update the release status to what's found in the release registers


In [92]:
import pandas as pd
import numpy as np
import os
import re
import io
import xlwings as xw
import openpyxl
from pathlib import Path
import argparse
import platform
import sys


In [2]:
def type_of_script():
    '''
        determine where this script is running
        return either jupyter, ipython, terminal
    '''
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return 'jupyter'
        if 'terminal' in ipy_str:
            return 'ipython'
    except:
        return 'terminal'

In [3]:
def find_files():
    if 'macOS' in platform.platform():
        # set some defaults for testing on mac
        download_dir = Path('/Users/mark/Downloads')
        user_dir = download_dir
        sharepoint_dir = download_dir

    elif 'Server' in platform.platform():
        # we're on the azure server (probably)
        user_dir = Path('Z:/python/FilesIn')

        download_dir = Path(user_dir)
        user_dir = download_dir
        sharepoint_dir = Path('Z:/python/FilesOut')

    elif os.getlogin() == 'mark_':
        # my test windows machine
        download_dir = Path('C:/Users/mark_/Downloads')
        user_dir = download_dir
        sharepoint_dir = download_dir        

    else:
        # personal one drive
        user_dir = 'C:/Users/USERNAME'

        # replace USERNAME with current logged on user
        user_dir = user_dir.replace('USERNAME', os.getlogin())

        # read in config file
        config = configparser.ConfigParser()
        config.read('user_directory.ini')

        # read in gm_dir and gm_docs from config file
        gm_dir = Path(config[os.getlogin().lower()]['gm_dir'])
        gm_docs = Path(config[os.getlogin().lower()]['gmt'])
        # this may find more than one sharepoint directory
        # sharepoint_dir = user_dir + "/" + gm_dir + "/" + gm_docs
        sharepoint_dir = Path(user_dir / gm_dir / gm_docs)

        # download_dir = os.path.join(sharepoint_dir, 'Data Shuttle', 'downloads')
        download_dir = Path(sharepoint_dir / 'Data Shuttle' / 'downloads')

    # find any changed files changed in past 2hrs in the downloads directory
    dirpath = download_dir
    files = []
    for p, ds, fs in os.walk(dirpath):
        for fn in fs:
            if 'Updated_' in fn:
                # was using this to filter what filenames to find
                filepath = os.path.join(p, fn)
                files.append(filepath)

    return files

In [4]:
def remove_cols(df, col_name):
    cols = df.columns

    # filter out all Unnamed cols
    cols = [ x for x in cols if col_name not in x ]

    df = df[cols]

    return df

# Makes without Buys
Each Make part needs at least one Buy part below it to make sense

If a MAKE is not followed by a child BUY this is a problem

In [131]:
def check_make_no_buy(df):
    # if MAKE and there is no BUY below next row's part level less than or equal to current part level we have a MAKE without a BUY
    # df['PROVIDE'] = np.where(df['Source Code'].isin(['AIH','MIH','MOB']),'Make','Buy')
    make_no_buy = list(df[(df['Source Code'].isin(['AIH','MIH','MOB'])) & (df['Level'].shift(-1) <= df['Level'])].index)
    make_no_buy = sorted(make_no_buy)
    df['make_no_buy'] = np.where((df['Source Code'].isin(['AIH','MIH','MOB'])) & (df['Level'].shift(-1) <= df['Level']), True, False)

    return df, make_no_buy

In [127]:
def parent_source_code(df):
    prev_level = 0

    level_source = {}

    for i, x in df.iterrows():
        # take the current level source and store it
        level_source[x['Level']] = x['Source Code']
        if ((x['Level'] >= 4)):
            df.loc[i, 'Parent Source Code'] = level_source[x['Level'] - 1]
    
    return df

In [79]:
def source_code_checks(df, sc_check_list):

    dict_checks = {}

    fastener_list = ['^WASHER|^BOLT|^GROMMET']
    for sc_check in sc_check_list:
        sc, parent_sc = sc_check.split('_')

        dict_checks[sc_check] = df[(df['Source Code'] == sc) & (df['Parent Source Code'] == parent_sc)]

        df[sc_check] = np.where((df['Source Code'] == sc) & (df['Parent Source Code'] == parent_sc), True, False)

        # temp_df = df[check_columns][(df['Description'].str.contains('^{}'.format(fastener))) & (df['Source Code'] == 'BOF')]
        # df['FAS_as_BOF'] = np.where((df['Description'].str.contains('^{}'.format(fastener))) & (df['Source Code'] == 'BOF'), True, False)

        # dict_checks['FAS_as_BOF'] = pd.concat([dict_checks['Fasteners_as_BOF'], temp_df])

    dict_checks['Non_MIH_AIH_Level_4'] = df[(df['Level'] == 4) & (~df['Source Code'].isin(['MIH','AIH']))]
    df['FAS_as_BOF'] = np.where((df['Level'] == 4) & (df['Source Code'] == 'BOF'), True, False)

    dict_checks['FAS_Wrong_Parent_Source_code'] = df[(df['Source Code'] == 'FAS') & (~df['Parent Source Code'].isin(['FIP','AIH','MIH']))]
    df['FAS_Wrong_Parent_Source_code'] = np.where((df['Source Code'] == 'FAS') & (~df['Parent Source Code'].isin(['FIP','AIH','MIH'])), True, False)

    return dict_checks, df


In [90]:
def fastener_checks(dict_checks, df, fastener_check_list):
    # All BOF records that are fasteners should be {FAS}teners in the BOMS
    # Part Description contains washer, bolt, grommet
    # Source code = "BOF"
        
    dict_checks['FAS_as_BOF'] = df[(df['Description'].str.lower().str.contains('{}'.format(fastener_check_list))) & (df['Source Code'] == 'BOF')]
    df['FAS_as_BOF'] = np.where((df['Description'].str.lower().str.contains('{}'.format(fastener_check_list))) & (df['Source Code'] == 'BOF'), True, False)

    return dict_checks, df

In [119]:
def filter_check_columns(dict_checks):
    # For a selection of columns, create a dataframe of AIH parent, POA parts
    check_columns = [
    # 'row',
    'Function Group',
    'System',
    'Sub System',
    'Level',
    'Title',
    'Revision',
    'Description',
    'Parent Part',
    'Source Code',
    'Quantity',
    'Parent Source Code'
    ]

    for key in dict_checks.keys():
        print (key)
        dict_checks[key] = dict_checks[key][check_columns]

    return dict_checks

In [167]:
def write_to_xl(outfile, df_dict):
    import xlwings as xw

    with xw.App(visible=True) as app:
        try:
            wb = xw.Book(outfile)
        except FileNotFoundError:
            wb = xw.Book()
            wb.save(outfile)

        for key in df_dict.keys():
            try:
                ws = wb.sheets.add(key)
            except Exception as e:
                print (e)
            
            ws = wb.sheets[key]

            table_name = key

            ws.clear()

            df = df_dict[key].set_index(list(df_dict[key])[0])
            if table_name in [table.df for table in ws.tables]:
                ws.tables[table_name].update(df)
            else:
                table_name = ws.tables.add(source=ws['A1'],
                                            name=table_name).update(df)
    wb.save(outfile)

In [109]:
def write_to_xl_sub_system(dict_checks):
    sub_sys = dict_checks[check]['Sub System'].unique()
    sub_sys.sort()

    for s_sys in sub_sys:

        df_temp = dict_checks[check][dict_checks[check]['Sub System'] == s_sys]

        if df_temp.shape[0] > 0:
            df_temp.to_excel(writer, sheet_name=s_sys, index=False)

            ws = writer.sheets[s_sys]
            wb = writer.book

            excel_formatting.adjust_col_width_from_col(ws)

# Main Processing

In [165]:
if __name__ == '__main__':

    # for reading in multiple files

    # files = find_files()
    dict_df = {}

    file = Path('/Users/mark/Downloads/Updated_T48e-01-Z00001_2024-07-19.xlsx')

    df = pd.DataFrame()
    existing_bom = pd.DataFrame()

    with open(file, "rb") as f:
        # reading in the historic excel files
        df = pd.read_excel(f, parse_dates=True)
        f.close()

    df.reset_index(drop=False, inplace=True)
    df.rename(columns={'index':'bom_order'}, inplace=True)

    # add parent source code to each row for validation checks to come
    df = parent_source_code(df)

    sc_check_list = ['AIH_POA','BOP_FIP','FAS_FAS','FIP_FIP','FIP_FAS']
    dict_checks, df = source_code_checks(df, sc_check_list)


    fastener_check_list = ['^washer|^bolt|^grommet']
    dict_checks, df = fastener_checks(dict_checks, df, fastener_check_list)

    df, make_no_buy = check_make_no_buy(df)

    dict_checks['make_no_buy'] = df.loc[make_no_buy]

    dict_checks = filter_check_columns(dict_checks)



AIH_POA
BOP_FIP
FAS_FAS
FIP_FIP
FIP_FAS
Non_MIH_AIH_Level_4
FAS_Wrong_Parent_Source_code
FAS_as_BOF
make_no_buy


### Release status checks
For part levels >=4 groupby part number and release status and report which parts have more than 1 release status - should be unique per part number

| 	|	|	|Function Group|	Part Level|	Issue Level|	row|
|---|---|---|--------------|--------------|------------|-------|
|Part Number|	Sub Group|	Release Status|||||				
|T33-A1117X	|A01-Structure Systems	|REL|	T33-BoM-XP	|7.0	|1.0	|617|
|||AWT	|T33-BoM-XP	|6.0	|1.0	|1284|
|T33-A1475	|A02-Panels & Closure Systems	|AWT	|T33-BoM-XP	|6.0	|1.0	|137|
|||REL	|T33-BoM-XP	|6.0	|1.0	|230|
|T33-A1476X	|A02-Panels & Closure Systems	|AWT	|T33-BoM-XP	|6.0	|1.0	|143|


This is written out to excel


### Write out the validation checks to excel

- collated bom multi status   
- wrong parent parts   
- collated and cleaned bom written out with cleaned part numbers, parent parts and release status attached   

This is where we can add a list of source code combinations to check as we find out they are not valid (mainly causing a problem for IFS)

### Write out source code checks to excel

In [168]:
# Write out to excel
pathfile = Path(file.name).stem
output_file = Path(sys.path[0]) / Path(pathfile + '_validated').with_suffix('.xlsx')

write_to_xl(output_file, dict_checks)

Command failed:
		OSERROR: -1728
		MESSAGE: The object you are trying to access does not exist
		COMMAND: app(pid=44727).workbooks['Updated_T48e-01-Z00001_2024-07-19_validated.xlsx'].sheets['Sheet2'].name.get()
Command failed:
		OSERROR: -1728
		MESSAGE: The object you are trying to access does not exist
		COMMAND: app(pid=44727).workbooks['Updated_T48e-01-Z00001_2024-07-19_validated.xlsx'].sheets['Sheet3'].name.get()
Command failed:
		OSERROR: -1728
		MESSAGE: The object you are trying to access does not exist
		COMMAND: app(pid=44727).workbooks['Updated_T48e-01-Z00001_2024-07-19_validated.xlsx'].sheets['Sheet4'].name.get()
Command failed:
		OSERROR: -1728
		MESSAGE: The object you are trying to access does not exist
		COMMAND: app(pid=44727).workbooks['Updated_T48e-01-Z00001_2024-07-19_validated.xlsx'].sheets['Sheet5'].name.get()
Command failed:
		OSERROR: -1728
		MESSAGE: The object you are trying to access does not exist
		COMMAND: app(pid=44727).workbooks['Updated_T48e-01-Z00001_2

CommandError: Command failed:
		OSERROR: -600
		MESSAGE: Application isn't running.
		COMMAND: app(pid=44727).display_alerts.get()

In [91]:
import xlwings as xw
from openpyxl.utils.cell import get_column_letter

make_filename = os.path.join(sharepoint_dir, project, project + '_MakeBuyQuery.xlsm')

with xw.App():
    try:
        wb = xw.Book(make_filename)
    except:
        wb = xw.Book()
    
    ws = wb.sheets[0]

    startrow=0

    ws.range("A:XX").clear()

    ws['A1'].options(pd.DataFrame, header=1, index=False).value=existing_bom

    last_col_letter = get_column_letter(existing_bom.shape[1])


    for row in make_no_buy:
        xw.Range('A{}:{}{}'.format(row + 2, last_col_letter, row + 2)).color = (69, 165, 237)

    ws.tables.add(ws.used_range, name="a_table")

    # autofit the columns
    ws.autofit('c')
    wb.save(make_filename)


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', "Cannot access 'T50s_MakeBuyQuery.xlsm'.", 'xlmain11.chm', 0, -2146827284), None)

# Multiple Source Codes

In [52]:
def multi_source_code(df):

    unstacked = df.groupby(['Part Number','Issue Level','Function Group','Sub Group','Source Code']).size().unstack()

    # find number of columns dynamically, as number of unique status controls the number of columns
    expected_status_count = len(unstacked.columns) - 1
    unstacked2 = unstacked[unstacked.isna().sum(axis=1)!=expected_status_count]
    unstacked2


    multi_sc = unstacked2.reset_index().fillna('')

    make_sc_cols = ['AIH','MIH','MOB']

    first_cols = ['Part Number', 'Issue Level', 'Function Group', 'Sub Group']

    cols_to_order = first_cols + make_sc_cols
    sc_ordered_cols = cols_to_order + (multi_sc.columns.drop(cols_to_order).tolist())

    multi_sc = multi_sc[sc_ordered_cols]

    return multi_sc


In [53]:
multi_sc = multi_source_code(existing_bom)

In [55]:
import xlwings as xw

multi_sc_filename = os.path.join(base, '{}-Multi-Source-Codes.xlsm'.format(project))

# wb = xw.Book(path)

# open file if it exists
try:
    wb = xw.Book(multi_sc_filename)
# otherwise create a new file    
except:
    wb = xw.Book()

ws = wb.sheets[0]

ws.clear_contents()

# autofit the columns
wb.sheets[ws].autofit('c')

startrow=0

ws['A1'].options(pd.DataFrame, header=1, index=False).value=multi_sc

# wb.save(multi_sc_filename) - this errors - not sure it's needed with autosave
